## Read Unseen DATA Watershed

In [3]:
import numpy as np

# Drop rows with any missing values
data_cleaned = df.dropna()
# Transform the 'E.coli' column to log10 scale
data_cleaned['Log10_E.coli'] = np.log10(data_cleaned['E.coli'])

# Display the first few rows to confirm the transformation
data_cleaned[['E.coli', 'Log10_E.coli']].head()

,E.coli,Log10_E.coli
0,82,1.913814
1,88,1.944483
2,120,2.079181
3,54,1.732394
4,40,1.602060


In [4]:
import pandas as pd
import numpy as np

# Load the Study Area 3 validation data
new_data = pd.read_excel('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario 0/Validation_DATA(StudyArea3) Scenario0Data.xlsx')

# Create 'Log10_E.coli' column
new_data['Log10_E.coli'] = np.log10(new_data['E.coli'])

# Save the updated file if you want
new_data.to_excel('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario 0/Validation_DATA(StudyArea3)_with_log10.xlsx', index=False)


## Pull Saved Model to test zero-shot

In [5]:
import pandas as pd
import numpy as np
import joblib

# === Load model and feature list ===
model = joblib.load('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario 0/ETR_ecoli_model.pkl')
final_features = joblib.load('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario 0/final_features.pkl')

# === Load new dataset ===
new_data = pd.read_excel('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario 0/Validation_DATA(StudyArea3)_with_log10.xlsx')

# Ensure all needed features are present
missing_features = [feat for feat in final_features if feat not in new_data.columns]
if missing_features:
    raise ValueError(f"Missing features in new data: {missing_features}")

# === Apply log1p transformation (consistent with training) ===
X_new_log1p = np.log1p(new_data[final_features].where(new_data[final_features] > 0))
X_new_log1p = X_new_log1p.replace([np.inf, -np.inf], np.nan).fillna(0)

# === Run prediction ===
y_new_pred = model.predict(X_new_log1p)

# === Output or save predictions ===
new_data['Predicted_Value'] = y_new_pred

# Save results
new_data.to_csv('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario2/new_data_with_predictions.csv', index=False)


## Test Zero-shot

In [8]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

# === Load model and feature list ===
model = joblib.load('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario 0/ETR_ecoli_model.pkl')
final_features = joblib.load('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario 0/final_features.pkl')

# === Load new dataset ===
new_data = pd.read_excel('C:/Users/as22dt/OneDrive - Florida State University/Analysis/Data/Scenario 0/Validation_DATA(StudyArea3)_with_log10.xlsx')

# === Prepare Data ===
# Ensure 'Date' is datetime
new_data['Date'] = pd.to_datetime(new_data['Date'])
new_data['Year'] = new_data['Date'].dt.year

# Create 'Log10_E.coli' if only raw E.coli is present
if 'Log10_E.coli' not in new_data.columns and 'E.coli' in new_data.columns:
    new_data['Log10_E.coli'] = np.log10(new_data['E.coli'])

# Check for missing features
missing_features = [feat for feat in final_features if feat not in new_data.columns]
if missing_features:
    raise ValueError(f"Missing features: {missing_features}")

# Apply log1p transformation to selected features
X_all_log1p = np.log1p(new_data[final_features].where(new_data[final_features] > 0))
X_all_log1p = X_all_log1p.replace([np.inf, -np.inf], np.nan).fillna(0)

# Target variable
target = 'Log10_E.coli'
y_all_true = new_data[target]

    # Predict
    y_interval_pred = model.predict(X_interval)

    # Calculate metrics
    r2 = r2_score(y_interval_true, y_interval_pred)
    mse = mean_squared_error(y_interval_true, y_interval_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_interval_true, y_interval_pred)
    mape = mean_absolute_percentage_error(y_interval_true, y_interval_pred)

    print(f"\n Zero-shot Prediction:")
   # print(f"Samples: {sample_count}")
    print(f"R²: {r2:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.4f}")

    # Save observed vs predicted to Excel
    output_df = pd.DataFrame({
        'Date': date_interval.values,
        'Observed_Log10_E.coli': y_interval_true.values,
        'Predicted_Log10_E.coli': y_interval_pred,
        'Observed_E.coli': 10 ** y_interval_true.values,
        'Predicted_E.coli': 10 ** y_interval_pred
    })

    output_path = 'observed_vs_predicted_custom_interval.xlsx'
    with pd.ExcelWriter(output_path, engine='xlsxwriter', datetime_format='yyyy-mm-dd') as writer:
        output_df.to_excel(writer, sheet_name='Results', index=False)
    print(f"\nObserved vs. Predicted values saved to '{output_path}'.")



 Zero-shot Prediction:
R²: 0.6305
MSE: 0.1457
RMSE: 0.3817
MAE: 0.3023
MAPE: 0.1373

Observed vs. Predicted values saved to 'observed_vs_predicted_custom_interval.xlsx'.
